# 🔴 Avance | ⏱ 45 min | 🔑 Concepts : duck typing, ABC, Protocol, polymorphisme

# Polymorphisme et Duck Typing en Python

## Objectifs

À la fin de ce notebook, vous serez capable de :
- Comprendre le polymorphisme en Python
- Maîtriser le duck typing ("If it walks like a duck...")
- Utiliser les ABC (Abstract Base Classes) avec `abc` module
- Créer des méthodes abstraites avec `@abstractmethod`
- Utiliser `Protocol` pour le structural subtyping (Python 3.8+)
- Distinguer interfaces implicites et explicites
- Comprendre la surcharge d'opérateurs

## Prérequis

- Connaissance de l'héritage et du MRO
- Compréhension des classes et méthodes

## 1. Polymorphisme en Python

Le **polymorphisme** permet d'utiliser des objets de différentes classes de manière uniforme, tant qu'ils partagent une interface commune.

En Python, le polymorphisme est **naturel** grâce au typage dynamique : on ne vérifie pas le type, seulement que l'objet possède les méthodes nécessaires.

In [ ]:
class Chien:
    def __init__(self, nom):
        self.nom = nom
    
    def faire_du_bruit(self):
        return "Woof!"

class Chat:
    def __init__(self, nom):
        self.nom = nom
    
    def faire_du_bruit(self):
        return "Miaou!"

class Vache:
    def __init__(self, nom):
        self.nom = nom
    
    def faire_du_bruit(self):
        return "Meuh!"

# Fonction polymorphe : fonctionne avec n'importe quel objet ayant faire_du_bruit()
def faire_parler(animal):
    print(f"{animal.nom} dit : {animal.faire_du_bruit()}")

# Utilisation
animaux = [
    Chien("Rex"),
    Chat("Félix"),
    Vache("Marguerite")
]

for animal in animaux:
    faire_parler(animal)  # Polymorphisme en action

## 2. Duck Typing : "If it walks like a duck..."

Le **duck typing** est un principe Python fondamental :

> "If it walks like a duck and quacks like a duck, it must be a duck."

On ne vérifie pas le **type** d'un objet, mais ses **capacités** (méthodes et attributs).

### Philosophie
- Pas besoin d'héritage commun
- Pas besoin d'interfaces explicites
- Si l'objet a les bonnes méthodes, il fonctionne

In [ ]:
# Duck typing : pas besoin de classe de base commune
class Fichier:
    def __init__(self, nom):
        self.nom = nom
        self.contenu = ""
    
    def write(self, texte):
        self.contenu += texte
    
    def read(self):
        return self.contenu

class Logger:
    def __init__(self):
        self.logs = []
    
    def write(self, message):
        from datetime import datetime
        self.logs.append(f"[{datetime.now()}] {message}")
    
    def read(self):
        return "\n".join(self.logs)

class MemoryBuffer:
    def __init__(self):
        self.buffer = []
    
    def write(self, data):
        self.buffer.append(data)
    
    def read(self):
        return "".join(self.buffer)

# Fonction qui accepte n'importe quel objet "file-like"
def sauvegarder_donnees(destination, donnees):
    """Duck typing : destination doit avoir write() et read()"""
    for donnee in donnees:
        destination.write(str(donnee) + "\n")
    print(f"Données sauvegardées dans {destination.__class__.__name__}")

# Utilisation avec différents types
donnees = ["Ligne 1", "Ligne 2", "Ligne 3"]

fichier = Fichier("data.txt")
sauvegarder_donnees(fichier, donnees)
print(f"Contenu fichier :\n{fichier.read()}\n")

logger = Logger()
sauvegarder_donnees(logger, donnees)
print(f"Logs :\n{logger.read()}\n")

buffer = MemoryBuffer()
sauvegarder_donnees(buffer, donnees)
print(f"Buffer :\n{buffer.read()}")

### EAFP vs LBYL

Le duck typing s'accompagne du principe **EAFP** (Easier to Ask for Forgiveness than Permission) plutôt que **LBYL** (Look Before You Leap).

In [ ]:
# ❌ LBYL (Look Before You Leap) - Non Pythonic
def traiter_lbyl(obj):
    if hasattr(obj, 'write') and callable(getattr(obj, 'write')):
        obj.write("test")
    else:
        print("Objet n'a pas de méthode write")

# ✅ EAFP (Easier to Ask for Forgiveness than Permission) - Pythonic
def traiter_eafp(obj):
    try:
        obj.write("test")
    except AttributeError:
        print("Objet n'a pas de méthode write")

# Test
class AvecWrite:
    def write(self, data):
        print(f"Écrit : {data}")

class SansWrite:
    pass

print("LBYL :")
traiter_lbyl(AvecWrite())
traiter_lbyl(SansWrite())

print("\nEAFP :")
traiter_eafp(AvecWrite())
traiter_eafp(SansWrite())

## 3. ABC (Abstract Base Classes)

Les **ABC** permettent de définir des **classes abstraites** qui ne peuvent pas être instanciées directement. Elles servent de contrat pour les sous-classes.

### Module `abc`
- `ABC` : classe de base pour les classes abstraites
- `@abstractmethod` : décore les méthodes qui doivent être implémentées

In [ ]:
from abc import ABC, abstractmethod

class Forme(ABC):  # Hérite de ABC
    """Classe abstraite pour les formes géométriques"""
    
    def __init__(self, couleur):
        self.couleur = couleur
    
    @abstractmethod
    def aire(self):
        """Méthode abstraite : doit être implémentée par les sous-classes"""
        pass
    
    @abstractmethod
    def perimetre(self):
        """Méthode abstraite"""
        pass
    
    # Méthode concrète (non abstraite)
    def description(self):
        return f"Forme de couleur {self.couleur}"

# ❌ Impossible d'instancier une classe abstraite
try:
    forme = Forme("rouge")
except TypeError as e:
    print(f"Erreur : {e}\n")

# ✅ Créer une sous-classe concrète
class Rectangle(Forme):
    def __init__(self, couleur, largeur, hauteur):
        super().__init__(couleur)
        self.largeur = largeur
        self.hauteur = hauteur
    
    # Implémentation obligatoire
    def aire(self):
        return self.largeur * self.hauteur
    
    def perimetre(self):
        return 2 * (self.largeur + self.hauteur)

rect = Rectangle("bleu", 10, 5)
print(f"Rectangle : {rect.description()}")
print(f"Aire : {rect.aire()}")
print(f"Périmètre : {rect.perimetre()}")

In [ ]:
# ❌ Oublier d'implémenter une méthode abstraite
class Cercle(Forme):
    def __init__(self, couleur, rayon):
        super().__init__(couleur)
        self.rayon = rayon
    
    def aire(self):
        import math
        return math.pi * self.rayon ** 2
    
    # ❌ Oubli d'implémenter perimetre()

try:
    cercle = Cercle("rouge", 5)
except TypeError as e:
    print(f"Erreur : {e}")
    print("La méthode abstraite perimetre() n'a pas été implémentée")

## 4. `@abstractmethod` : forcer l'implémentation

`@abstractmethod` garantit que les sous-classes implémentent certaines méthodes.

In [ ]:
from abc import ABC, abstractmethod

class BaseDeDonnees(ABC):
    """Interface abstraite pour une base de données"""
    
    @abstractmethod
    def connecter(self):
        """Établir la connexion"""
        pass
    
    @abstractmethod
    def executer(self, requete):
        """Exécuter une requête"""
        pass
    
    @abstractmethod
    def fermer(self):
        """Fermer la connexion"""
        pass

class PostgreSQL(BaseDeDonnees):
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.connecte = False
    
    def connecter(self):
        print(f"Connexion à PostgreSQL {self.host}:{self.port}")
        self.connecte = True
    
    def executer(self, requete):
        if not self.connecte:
            raise Exception("Non connecté")
        print(f"Exécution : {requete}")
        return ["résultat1", "résultat2"]
    
    def fermer(self):
        print("Fermeture de la connexion PostgreSQL")
        self.connecte = False

class MongoDB(BaseDeDonnees):
    def __init__(self, uri):
        self.uri = uri
        self.connecte = False
    
    def connecter(self):
        print(f"Connexion à MongoDB {self.uri}")
        self.connecte = True
    
    def executer(self, requete):
        if not self.connecte:
            raise Exception("Non connecté")
        print(f"Exécution : {requete}")
        return {"documents": [1, 2, 3]}
    
    def fermer(self):
        print("Fermeture de la connexion MongoDB")
        self.connecte = False

# Fonction polymorphe
def utiliser_base(db: BaseDeDonnees):
    db.connecter()
    resultats = db.executer("SELECT * FROM users")
    print(f"Résultats : {resultats}")
    db.fermer()

# Utilisation
print("PostgreSQL :")
pg = PostgreSQL("localhost", 5432)
utiliser_base(pg)

print("\nMongoDB :")
mongo = MongoDB("mongodb://localhost:27017")
utiliser_base(mongo)

### Méthodes abstraites avec implémentation par défaut

In [ ]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def faire_du_bruit(self):
        """Méthode abstraite avec implémentation par défaut"""
        return "[bruit générique]"
    
    def description(self):
        return f"Un animal qui fait : {self.faire_du_bruit()}"

class Chien(Animal):
    def faire_du_bruit(self):
        # Peut appeler super() pour étendre l'implémentation par défaut
        return "Woof!"

class Chat(Animal):
    def faire_du_bruit(self):
        bruit_generique = super().faire_du_bruit()
        return f"Miaou! {bruit_generique}"

chien = Chien()
chat = Chat()

print(chien.description())
print(chat.description())

## 5. Protocol (Python 3.8+) : structural subtyping

`Protocol` (du module `typing`) permet de définir des **interfaces structurelles** : un objet est compatible s'il a les bonnes méthodes, **sans héritage explicite**.

C'est du duck typing **avec vérification de type statique**.

In [ ]:
from typing import Protocol

class Drawable(Protocol):
    """Protocol : tout objet avec draw() est Drawable"""
    def draw(self) -> str:
        ...

# Classes qui implémentent implicitement Drawable (sans héritage)
class Cercle:
    def __init__(self, rayon):
        self.rayon = rayon
    
    def draw(self) -> str:
        return f"○ Cercle (rayon={self.rayon})"

class Rectangle:
    def __init__(self, largeur, hauteur):
        self.largeur = largeur
        self.hauteur = hauteur
    
    def draw(self) -> str:
        return f"▭ Rectangle ({self.largeur}x{self.hauteur})"

class Texte:
    def __init__(self, contenu):
        self.contenu = contenu
    
    def draw(self) -> str:
        return f"✎ Texte: {self.contenu}"

# Fonction avec type hint Protocol
def afficher(obj: Drawable) -> None:
    """Accepte tout objet Drawable (duck typing + type hint)"""
    print(obj.draw())

# Utilisation : pas besoin d'héritage !
objets = [
    Cercle(5),
    Rectangle(10, 20),
    Texte("Hello, World!")
]

for obj in objets:
    afficher(obj)  # Type checker valide que obj a draw()

### Protocol avec plusieurs méthodes

In [ ]:
from typing import Protocol

class Comparable(Protocol):
    """Protocol pour les objets comparables"""
    def __lt__(self, other) -> bool:
        ...
    
    def __eq__(self, other) -> bool:
        ...

class Personne:
    def __init__(self, nom: str, age: int):
        self.nom = nom
        self.age = age
    
    def __lt__(self, other: 'Personne') -> bool:
        return self.age < other.age
    
    def __eq__(self, other: 'Personne') -> bool:
        return self.age == other.age
    
    def __repr__(self):
        return f"Personne('{self.nom}', {self.age})"

class Produit:
    def __init__(self, nom: str, prix: float):
        self.nom = nom
        self.prix = prix
    
    def __lt__(self, other: 'Produit') -> bool:
        return self.prix < other.prix
    
    def __eq__(self, other: 'Produit') -> bool:
        return self.prix == other.prix
    
    def __repr__(self):
        return f"Produit('{self.nom}', {self.prix}€)"

# Fonction générique pour trier n'importe quel Comparable
def trier_et_afficher(items: list[Comparable]) -> None:
    tries = sorted(items)
    for item in tries:
        print(item)

# Utilisation
personnes = [
    Personne("Alice", 30),
    Personne("Bob", 25),
    Personne("Charlie", 35)
]

produits = [
    Produit("Laptop", 999.99),
    Produit("Souris", 29.99),
    Produit("Clavier", 79.99)
]

print("Personnes triées par âge :")
trier_et_afficher(personnes)

print("\nProduits triés par prix :")
trier_et_afficher(produits)

## 6. `typing.Protocol` vs `ABC`

### Quand utiliser `Protocol` ?
- Duck typing avec vérification de type statique
- Pas besoin d'héritage explicite
- Intégration avec du code existant
- Type hints pour les type checkers (mypy, pyright)

### Quand utiliser `ABC` ?
- Contrôle strict à l'exécution
- Hiérarchie de classes explicite
- Empêcher l'instanciation de la classe de base
- Documentation claire de l'héritage

In [ ]:
from abc import ABC, abstractmethod
from typing import Protocol

# Approche ABC : héritage explicite
class FichierABC(ABC):
    @abstractmethod
    def read(self) -> str:
        pass
    
    @abstractmethod
    def write(self, data: str) -> None:
        pass

class FichierTexte(FichierABC):  # Doit hériter
    def __init__(self, nom):
        self.nom = nom
        self.contenu = ""
    
    def read(self) -> str:
        return self.contenu
    
    def write(self, data: str) -> None:
        self.contenu += data

# Approche Protocol : pas d'héritage nécessaire
class FichierProtocol(Protocol):
    def read(self) -> str:
        ...
    
    def write(self, data: str) -> None:
        ...

class BufferMemoire:  # Pas d'héritage !
    def __init__(self):
        self.buffer = ""
    
    def read(self) -> str:
        return self.buffer
    
    def write(self, data: str) -> None:
        self.buffer += data

# Fonction avec ABC : vérifie l'héritage
def traiter_abc(fichier: FichierABC) -> None:
    assert isinstance(fichier, FichierABC), "Doit hériter de FichierABC"
    fichier.write("test")
    print(fichier.read())

# Fonction avec Protocol : vérifie la structure
def traiter_protocol(fichier: FichierProtocol) -> None:
    # Pas de vérification d'héritage, juste la structure
    fichier.write("test")
    print(fichier.read())

# Tests
ft = FichierTexte("test.txt")
bm = BufferMemoire()

print("ABC :")
traiter_abc(ft)  # ✅ OK
try:
    traiter_abc(bm)  # ❌ Erreur : pas d'héritage
except AssertionError as e:
    print(f"Erreur : {e}")

print("\nProtocol :")
traiter_protocol(ft)  # ✅ OK
traiter_protocol(bm)  # ✅ OK (duck typing)

## 7. Interfaces implicites vs explicites

Python privilégie les **interfaces implicites** (duck typing) mais supporte aussi les **interfaces explicites** (ABC).

In [ ]:
# Interface implicite (Pythonic)
class IterateurImplicite:
    """Pas besoin de déclarer qu'on implémente un itérateur"""
    def __init__(self, data):
        self.data = data
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration
        valeur = self.data[self.index]
        self.index += 1
        return valeur

# Utilisation : fonctionne automatiquement avec for
for item in IterateurImplicite([1, 2, 3]):
    print(item)

print("\n" + "="*50 + "\n")

# Interface explicite (avec ABC)
from abc import ABC, abstractmethod

class Iterator(ABC):
    @abstractmethod
    def __iter__(self):
        pass
    
    @abstractmethod
    def __next__(self):
        pass

class IterateurExplicite(Iterator):
    """Déclare explicitement implémenter Iterator"""
    def __init__(self, data):
        self.data = data
        self.index = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.index >= len(self.data):
            raise StopIteration
        valeur = self.data[self.index]
        self.index += 1
        return valeur

# Vérification explicite possible
it = IterateurExplicite([4, 5, 6])
print(f"isinstance(it, Iterator) : {isinstance(it, Iterator)}")

for item in it:
    print(item)

## 8. Surcharge d'opérateurs (aperçu)

La surcharge d'opérateurs est une forme de polymorphisme qui permet de définir le comportement des opérateurs (`+`, `-`, `*`, etc.) pour nos classes.

Note : Ce sujet sera approfondi dans le cours sur les méthodes spéciales.

In [ ]:
class Vecteur:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    # Surcharge de +
    def __add__(self, other):
        return Vecteur(self.x + other.x, self.y + other.y)
    
    # Surcharge de *
    def __mul__(self, scalaire):
        return Vecteur(self.x * scalaire, self.y * scalaire)
    
    # Surcharge de ==
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __repr__(self):
        return f"Vecteur({self.x}, {self.y})"

v1 = Vecteur(1, 2)
v2 = Vecteur(3, 4)

print(f"v1 = {v1}")
print(f"v2 = {v2}")
print(f"v1 + v2 = {v1 + v2}")  # __add__
print(f"v1 * 3 = {v1 * 3}")    # __mul__
print(f"v1 == v2 : {v1 == v2}")  # __eq__
print(f"v1 == Vecteur(1, 2) : {v1 == Vecteur(1, 2)}")

## Pièges courants

### Piège 1 : ABC trop rigide

In [ ]:
# ❌ ABC trop stricte limite la réutilisation
from abc import ABC, abstractmethod

class StockageABC(ABC):
    @abstractmethod
    def sauvegarder(self, cle, valeur):
        pass
    
    @abstractmethod
    def charger(self, cle):
        pass

# On veut utiliser un dict standard, mais il ne hérite pas de StockageABC
def utiliser_stockage_abc(stockage: StockageABC):
    assert isinstance(stockage, StockageABC)
    stockage.sauvegarder("cle", "valeur")

try:
    utiliser_stockage_abc({})  # ❌ dict ne hérite pas de StockageABC
except AssertionError:
    print("Erreur : dict ne peut pas être utilisé car il ne hérite pas de StockageABC")

print("\n" + "="*50 + "\n")

# ✅ Protocol est plus flexible
from typing import Protocol

class StockageProtocol(Protocol):
    def __setitem__(self, cle, valeur):
        ...
    
    def __getitem__(self, cle):
        ...

def utiliser_stockage_protocol(stockage: StockageProtocol):
    stockage["cle"] = "valeur"  # __setitem__
    return stockage["cle"]  # __getitem__

# ✅ dict fonctionne avec Protocol (duck typing)
resultat = utiliser_stockage_protocol({})
print(f"Résultat : {resultat}")

### Piège 2 : Protocol mal compris

In [ ]:
from typing import Protocol

# ❌ Penser que Protocol fait une vérification à l'exécution
class Volant(Protocol):
    def voler(self) -> str:
        ...

class Avion:
    def voler(self) -> str:
        return "L'avion vole"

class Voiture:
    def rouler(self) -> str:
        return "La voiture roule"

def faire_voler(obj: Volant):
    # ⚠️ Protocol est seulement pour les type checkers (mypy, pyright)
    # Python n'empêche pas l'exécution si la signature ne correspond pas
    print(obj.voler())

# ✅ Fonctionne
faire_voler(Avion())

# ⚠️ Python n'empêche PAS l'exécution, mais mypy signalerait l'erreur
try:
    faire_voler(Voiture())  # AttributeError à l'exécution
except AttributeError as e:
    print(f"\nErreur à l'exécution : {e}")
    print("Protocol ne fait pas de vérification à l'exécution !")

### Piège 3 : Overuse de `isinstance()`

In [ ]:
# ❌ Trop de isinstance() va à l'encontre du polymorphisme
def traiter_mauvais(obj):
    if isinstance(obj, Chien):
        print("C'est un chien")
        obj.aboyer()
    elif isinstance(obj, Chat):
        print("C'est un chat")
        obj.miauler()
    elif isinstance(obj, Vache):
        print("C'est une vache")
        obj.beugler()
    # ❌ Code fragile : il faut ajouter un cas pour chaque nouveau type

# ✅ Polymorphisme : une seule méthode commune
def traiter_bon(animal):
    # Suppose que tous les animaux ont faire_du_bruit()
    print(animal.faire_du_bruit())

# Encore mieux : avec Protocol ou ABC pour documenter l'interface

## Mini-exercices

### Exercice 1 : Interface Shape avec ABC

Créez une classe abstraite `Shape` avec :
- Méthodes abstraites : `aire()`, `perimetre()`
- Méthode concrète : `description()` qui utilise `aire()` et `perimetre()`

Implémentez `Carre` et `Triangle`.
Testez avec une fonction polymorphe.

In [ ]:
# Votre code ici


### Solution Exercice 1

In [ ]:
from abc import ABC, abstractmethod
import math

class Shape(ABC):
    """Classe abstraite pour les formes géométriques"""
    
    @abstractmethod
    def aire(self) -> float:
        """Calculer l'aire"""
        pass
    
    @abstractmethod
    def perimetre(self) -> float:
        """Calculer le périmètre"""
        pass
    
    def description(self) -> str:
        """Description utilisant les méthodes abstraites"""
        return (
            f"{self.__class__.__name__} - "
            f"Aire: {self.aire():.2f}, "
            f"Périmètre: {self.perimetre():.2f}"
        )

class Carre(Shape):
    def __init__(self, cote):
        self.cote = cote
    
    def aire(self) -> float:
        return self.cote ** 2
    
    def perimetre(self) -> float:
        return 4 * self.cote

class Triangle(Shape):
    def __init__(self, a, b, c):
        # Vérifier qu'on peut former un triangle
        if not (a + b > c and b + c > a and a + c > b):
            raise ValueError("Dimensions invalides pour un triangle")
        self.a = a
        self.b = b
        self.c = c
    
    def aire(self) -> float:
        # Formule de Héron
        s = self.perimetre() / 2
        return math.sqrt(s * (s - self.a) * (s - self.b) * (s - self.c))
    
    def perimetre(self) -> float:
        return self.a + self.b + self.c

# Fonction polymorphe
def analyser_formes(formes: list[Shape]):
    aire_totale = 0
    perimetre_total = 0
    
    print("Analyse des formes :")
    print("=" * 60)
    for forme in formes:
        print(forme.description())
        aire_totale += forme.aire()
        perimetre_total += forme.perimetre()
    
    print("=" * 60)
    print(f"Aire totale : {aire_totale:.2f}")
    print(f"Périmètre total : {perimetre_total:.2f}")

# Tests
formes = [
    Carre(5),
    Triangle(3, 4, 5),
    Carre(10),
    Triangle(6, 8, 10)
]

analyser_formes(formes)

### Exercice 2 : Protocol pour fichiers

Créez un `Protocol` `Fichier` avec méthodes `read()` et `write()`.
Implémentez deux classes sans héritage :
- `FichierTexte` : stockage en string
- `FichierBinaire` : stockage en bytes

Créez une fonction polymorphe qui fonctionne avec les deux.

In [ ]:
# Votre code ici


### Solution Exercice 2

In [ ]:
from typing import Protocol, Union

class Fichier(Protocol):
    """Protocol pour les objets fichier"""
    def read(self) -> Union[str, bytes]:
        ...
    
    def write(self, data: Union[str, bytes]) -> None:
        ...

class FichierTexte:
    """Fichier texte (string)"""
    def __init__(self, nom: str):
        self.nom = nom
        self._contenu = ""
    
    def read(self) -> str:
        return self._contenu
    
    def write(self, data: str) -> None:
        if not isinstance(data, str):
            raise TypeError("FichierTexte attend du str")
        self._contenu += data
    
    def __repr__(self):
        return f"FichierTexte('{self.nom}')"

class FichierBinaire:
    """Fichier binaire (bytes)"""
    def __init__(self, nom: str):
        self.nom = nom
        self._contenu = b""
    
    def read(self) -> bytes:
        return self._contenu
    
    def write(self, data: bytes) -> None:
        if not isinstance(data, bytes):
            raise TypeError("FichierBinaire attend du bytes")
        self._contenu += data
    
    def __repr__(self):
        return f"FichierBinaire('{self.nom}')"

# Fonction polymorphe utilisant le Protocol
def copier_fichier(source: Fichier, destination: Fichier) -> None:
    """Copier le contenu de source vers destination"""
    contenu = source.read()
    destination.write(contenu)
    print(f"Copié de {source} vers {destination}")

def afficher_contenu(fichier: Fichier) -> None:
    """Afficher le contenu d'un fichier"""
    contenu = fichier.read()
    print(f"Contenu de {fichier} :")
    if isinstance(contenu, bytes):
        print(f"  (binaire) {contenu[:50]}...")  # Premiers 50 bytes
    else:
        print(f"  {contenu}")

# Tests
print("=== Fichiers texte ===")
ft1 = FichierTexte("source.txt")
ft1.write("Hello, World!\n")
ft1.write("Python est génial!\n")

ft2 = FichierTexte("destination.txt")
copier_fichier(ft1, ft2)
afficher_contenu(ft2)

print("\n=== Fichiers binaires ===")
fb1 = FichierBinaire("image.bin")
fb1.write(b"\x89PNG\r\n\x1a\n")
fb1.write(b"\x00\x00\x00\rIHDR")

fb2 = FichierBinaire("image_copie.bin")
copier_fichier(fb1, fb2)
afficher_contenu(fb2)

print("\n=== Mix (erreur attendue) ===")
try:
    copier_fichier(ft1, fb2)  # Type mismatch
except TypeError as e:
    print(f"Erreur : {e}")

### Exercice 3 : Duck typing en action

Créez 3 classes différentes **sans classe de base commune** :
- `Personne` : `nom`, méthode `presentation()`
- `Entreprise` : `nom`, méthode `presentation()`
- `Produit` : `nom`, méthode `presentation()`

Créez une fonction `afficher_catalogue()` qui fonctionne avec n'importe quel objet ayant `nom` et `presentation()`.

In [ ]:
# Votre code ici


### Solution Exercice 3

In [ ]:
class Personne:
    def __init__(self, nom: str, metier: str, age: int):
        self.nom = nom
        self.metier = metier
        self.age = age
    
    def presentation(self) -> str:
        return f"👤 {self.nom}, {self.age} ans, {self.metier}"

class Entreprise:
    def __init__(self, nom: str, secteur: str, employes: int):
        self.nom = nom
        self.secteur = secteur
        self.employes = employes
    
    def presentation(self) -> str:
        return f"🏢 {self.nom}, secteur {self.secteur}, {self.employes} employés"

class Produit:
    def __init__(self, nom: str, prix: float, stock: int):
        self.nom = nom
        self.prix = prix
        self.stock = stock
    
    def presentation(self) -> str:
        return f"📦 {self.nom}, {self.prix}€, stock: {self.stock}"

# Fonction utilisant duck typing
def afficher_catalogue(items):
    """
    Affiche un catalogue d'items.
    Duck typing : fonctionne avec tout objet ayant 'nom' et 'presentation()'
    """
    print("="*60)
    print(f"{'NOM':<30} | DÉTAILS")
    print("="*60)
    
    for item in items:
        # Duck typing : on suppose que item a 'nom' et 'presentation()'
        try:
            print(f"{item.nom:<30} | {item.presentation()}")
        except AttributeError as e:
            print(f"Erreur : {e}")
    
    print("="*60)

def rechercher_par_nom(items, nom: str):
    """Duck typing : recherche par nom"""
    resultats = [item for item in items if item.nom.lower() == nom.lower()]
    if resultats:
        print(f"\nTrouvé : {resultats[0].presentation()}")
    else:
        print(f"\nAucun résultat pour '{nom}'")

# Tests : catalogue mixte (duck typing en action)
catalogue = [
    Personne("Alice Dupont", "Développeuse", 30),
    Entreprise("TechCorp", "Informatique", 250),
    Produit("Laptop Pro", 1299.99, 15),
    Personne("Bob Martin", "Designer", 28),
    Produit("Souris Ergonomique", 49.99, 100),
    Entreprise("DataInc", "Big Data", 50)
]

afficher_catalogue(catalogue)

# Recherche polymorphe
rechercher_par_nom(catalogue, "TechCorp")
rechercher_par_nom(catalogue, "Laptop Pro")
rechercher_par_nom(catalogue, "Inconnu")

## Résumé

Dans ce notebook, vous avez appris :

- ✅ Le polymorphisme en Python est naturel grâce au typage dynamique
- ✅ Duck typing : "If it walks like a duck..." (pas besoin d'héritage)
- ✅ EAFP (Easier to Ask for Forgiveness than Permission) vs LBYL
- ✅ ABC (Abstract Base Classes) pour des interfaces explicites
- ✅ `@abstractmethod` pour forcer l'implémentation de méthodes
- ✅ `Protocol` pour le structural subtyping (duck typing + type hints)
- ✅ Différences entre `Protocol` (structural) et `ABC` (nominal)
- ✅ Interfaces implicites (Pythonic) vs explicites
- ✅ Surcharge d'opérateurs pour un polymorphisme naturel
- ✅ Éviter les pièges : ABC trop rigide, Protocol mal compris, overuse de isinstance()

**Concepts clés de la POO Python maîtrisés** :
1. Classes et instanciation
2. Constructeurs et destructeurs
3. Encapsulation et visibilité
4. Héritage simple et multiple
5. Polymorphisme et duck typing

**Prochaine étape** : Méthodes spéciales (`__str__`, `__repr__`, `__len__`, etc.)